In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import k-independent data
cells = pd.read_csv(f'data/cells_no_repeats.csv', index_col=0)
synapses = pd.read_csv(f'data/synapses_w_ids.csv', index_col=0)
synapses.set_index('synapse_id', inplace=True)
print('Cells:', cells.shape)
print('Synapses:', synapses.shape)
display(cells.head())
display(synapses.head())

Cells: (56209, 5)
Synapses: (5421809, 17)


,cell_type,pt_root_id,pt_x,pt_y,pt_z
Unnamed: 0,,,,,
0,23P,864691135639004475,71136,110528,20220
1,23P,864691135771677771,72576,108656,20291
2,23P,864691135864089470,79632,121456,16754
3,23P,864691135560505569,80128,124000,16563
4,23P,864691136315868311,80144,126928,16622


,pre_pt_root_id,post_pt_root_id,size,cell_type_pre,cb_x_pre,cb_y_pre,cb_z_pre,cell_type_post,cb_x_post,cb_y_post,cb_z_post,cb_x_diff,cb_y_diff,cb_z_diff,ctr_pt_x,ctr_pt_y,ctr_pt_z
synapse_id,,,,,,,,,,,,,,,,,
0,864691135564683351,864691136811959667,960,23P,557.248,570.560,732.52,23P,791.744,483.968,780.44,-234.496,86.592,-47.92,637.608,371.352,720.20
1,864691135614119115,864691135508912649,7576,23P,774.720,504.960,895.68,23P,807.936,459.584,870.28,-33.216,45.376,25.40,805.376,488.376,863.60
3,864691135113147801,864691136022555784,684,23P,883.072,451.456,817.84,23P,851.520,483.840,759.96,31.552,-32.384,57.88,858.328,516.648,775.88
4,864691135194393642,864691135341362885,23172,23P,781.248,449.984,696.88,23P,798.720,465.152,758.56,-17.472,-15.168,-61.68,789.400,478.040,691.00
5,864691136272938174,864691135683554546,3660,23P,762.368,473.792,773.68,23P,820.352,446.784,719.08,-57.984,27.008,54.60,756.624,440.928,710.60


In [20]:
# Specify k
k = 6

In [21]:
# Import k-dependent data
clusters = pd.read_csv(f'data/synapse_clusters/clusters_k{k}.csv', index_col=[0])
print(clusters.shape)
display(clusters.head())

(64330, 12)


,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5,density_score,post_pt_root_id,post_cell_type,centroid_x,centroid_y,centroid_z
cluster_id,,,,,,,,,,,,
0,941001,900422,906864,932560,932857,923889,0.397512,864691134884741370,4P,572.974667,691.796667,1052.113333
1,958004,979837,932857,932560,912217,906864,0.413748,864691134884741370,4P,577.322667,692.268000,1054.240000
2,674327,704519,662648,676483,709257,678862,0.384314,864691134884742906,6P-IT,488.466000,875.476000,936.273333
3,687488,689522,657710,670465,673155,692136,0.379345,864691134884742906,6P-IT,553.638667,1000.280667,1026.933333
4,685121,2241694,711425,686754,702105,676480,0.381143,864691134884743930,6P-IT,586.533333,758.324000,750.860000


In [22]:
unique_synapses = set()
for ij in range(k):
    unique_synapses.update(clusters[f'synapse_id_{ij}'])
print(len(unique_synapses))
print(clusters.shape[0] * k)

316685
385980


In [23]:
cluster_synapses = synapses.loc[list(unique_synapses)]
synapse_cntrs = cluster_synapses.loc[:, ['ctr_pt_x', 'ctr_pt_y', 'ctr_pt_z']]
print(cluster_synapses.shape)
print(synapse_cntrs.shape)
display(synapse_cntrs.head())

(316685, 17)
(316685, 3)


,ctr_pt_x,ctr_pt_y,ctr_pt_z
synapse_id,,,
3145728,928.336,788.120,1041.00
6291457,1402.388,871.196,762.48
4194308,1083.776,687.296,855.88
4194309,1052.504,728.680,1051.24
4194315,1032.672,782.016,895.40


In [24]:
id_keys = [f'synapse_id_{ij}' for ij in range(k)]
synapse_ids = clusters[id_keys]
display(synapse_ids.head())

,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5
cluster_id,,,,,,
0,941001,900422,906864,932560,932857,923889
1,958004,979837,932857,932560,912217,906864
2,674327,704519,662648,676483,709257,678862
3,687488,689522,657710,670465,673155,692136
4,685121,2241694,711425,686754,702105,676480


In [25]:
# Gather the x, y, z coordinates of each synapse in each cluster
col_names = [[f'synapse_ctr_x_{m}', f'synapse_ctr_y_{m}', f'synapse_ctr_z_{m}'] for m in range(k)]
for cluster_id, cur_synapse in synapse_ids.iterrows():
    for n in range(k):
        cur_id = cur_synapse[f'synapse_id_{n}']
        cur_syn_cntr = synapse_cntrs.loc[cur_id, :]
        col_name = col_names[n]
        clusters.loc[cluster_id, col_name] = cur_syn_cntr.values

print(clusters.shape)
display(clusters.head())

(64330, 30)


,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5,density_score,post_pt_root_id,post_cell_type,centroid_x,...,synapse_ctr_z_2,synapse_ctr_x_3,synapse_ctr_y_3,synapse_ctr_z_3,synapse_ctr_x_4,synapse_ctr_y_4,synapse_ctr_z_4,synapse_ctr_x_5,synapse_ctr_y_5,synapse_ctr_z_5
cluster_id,,,,,,,,,,,,,,,,,,,,,
0,941001,900422,906864,932560,932857,923889,0.397512,864691134884741370,4P,572.974667,...,1052.52,572.040,693.872,1058.16,578.620,694.676,1055.72,575.424,686.624,1045.36
1,958004,979837,932857,932560,912217,906864,0.413748,864691134884741370,4P,577.322667,...,1055.72,572.040,693.872,1058.16,579.460,687.196,1047.64,571.656,689.108,1052.52
2,674327,704519,662648,676483,709257,678862,0.384314,864691134884742906,6P-IT,488.466000,...,939.88,487.176,873.280,936.16,487.560,870.816,933.72,489.128,867.528,932.28
3,687488,689522,657710,670465,673155,692136,0.379345,864691134884742906,6P-IT,553.638667,...,1021.52,556.756,998.768,1031.00,558.924,998.200,1029.52,558.896,998.608,1030.56
4,685121,2241694,711425,686754,702105,676480,0.381143,864691134884743930,6P-IT,586.533333,...,747.52,586.616,765.336,747.80,588.208,753.416,755.12,588.112,751.376,754.92


In [26]:
clusters.to_csv(f'data/synapse_clusters/with_syn_ctrs/clusters_k{k}.csv')